<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2023notebooks/2022_0123ccap_lam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


* date: 2023_0123
* fname: 2022_0123ccap_lam.ipynb


* 一文字の orth2phon を担保したいために，全角の数字，アルファベット，ひらがな，計 109 文字をデータ先頭に追加した。
* Fushimi1999 (Psyc. Rev.) の語彙リストを fushimi1999_list として収録
* Fushimi1999_list の扱いに伴い訓練語彙数を 10K から 20K に増加
* 学習率 lr は 0.001 だと収束しない。0.0001 であれば良好であり，訓練損失 0.01 程度，訓練精度 0.987 程度までに至る。
* ただし，一文字データセット onechar_dataset では lr=0.001 の方が収束が早い。
これは，データセットサイズが 20K と 0.1K と 20 倍の差があるためであろう。
* 近藤先生が，GPU 上で実行してくださった訓練済モデルのファイル名が `decoder256new.pt` と `encoder256new.pt` である。
これは，中間層ユニット数が 256 である orth2phon モデルの訓練済モデルである。
* `_train()` 関数内で，正解判定をする際に，GPU から CPU へ転送しなければいけないことを忘れていたので修正した。
具体的には， `detach()` と `numpy()` の間に `cpu()` を挿入した。2 箇所
```python
    ok_flag = (ok_flag) and (decoder_output.argmax() == target_tensor[di].detach().cpu().numpy()[0])
```

近藤先生の実験によれば，結果は以下の通りである(そうだ)。

正答率

|   | 条件 | 記述         | 正解率 | 
|:----|:-----|:------------|:------|
|WORD |   HF |1:consistent |　18/20
|WORD |   HF |2:typical    |   HF___inconsist  16/20|
|WORD |   HF |3:atypical   |   HF___atypical_  8/20 |
|WORD |   LF |1:consistent |   LF___consist__  14/20|
|WORD |   LF |2:typical    |   LF___inconsist  9/20|
|WORD |   LF |3:atypical   |   LF___atypical_  3/20|

* 伏見らではでなかったatypical効果だけでなく，
　consistent-typicalの差もある程度ある気がします
 また，LFでも効果ありであり，かつ，頻度効果もあり
* **今回，L(legitimate alternative reading of components） マークを付けてみました**
  Lm, Lnは，モーラ間違い，一文字間違いと混合


アクセプト率

|     | 条件 | 記述         | 正解率 | 
|:----|:----|:------------|:------|
|非単語| HF  | 1:consistent|HFNW_consist__  17/20|
|非単語| HF  | 2:typical   |HFNW_inconsist　　17/20|
|非単語| HF  | 3:ambiguous |HFNW_ambiguous  13/20|
|非単語| LF  | 1:consistent|LFNW_consist__  15/20|
|非単語| LF  | 2:typical   |LFNW_inconsist  13/20|
|非単語| LF  | 3:ambiguous |LFNW_ambiguous  7/20|

* かなり読めますね．アクセプトは，どんな読みでもいいので読めそうな読み方ならOKにしています．
　単語の L と同じになります．
* **結構驚きは，非単語のときに連濁や促音化ができているところ**

* 2023_0116 関係者にわかりやうように，コメントを多用して，問題点を共有するように務めること

In [ ]:
# ここはお遊びなので，スキップしても良い
#import IPython
#IPython.display.Image(url="https://livedoor.blogimg.jp/ftb001/imgs/b/4/b4629a79.jpg")
#IPython.display.Image(url="https://uy-allstars.com/_assets/images/pages/char/detail/webp/lum@pc.webp")

import os
lum_img_fname = 'lum@pc.webp'
if not os.path.exists(lum_img_fname):
    !wget "https://uy-allstars.com/_assets/images/pages/char/detail/webp/lum@pc.webp"

import matplotlib.pyplot as plt
x = plt.imread('lum@pc.webp')
plt.figure(figsize=(4,8))
plt.imshow(x)

# 1 準備作業


## 1.1 ライブラリのインポート

In [ ]:
%config InlineBackend.figure_format = 'retina'
try:
    import bit
except ImportError:
    !pip install ipynbname --upgrade > /dev/null 2>&1 
    !git clone https://github.com/ShinAsakawa/bit.git 2>&1
    
    # `import bit` する前に termcolor を downgrade しないと colab では色付きテキスト不能
    !pip install --upgrade termcolor==2.0 2>&1  
    import termcolor    
    import bit

isColab = bit.isColab
HOME = bit.HOME

if isColab:
    # colab 上で MeCab を動作させるために，C コンパイラを起動して，MeCab の構築を行う
    # そのため時間がかかる。
    !apt install aptitude
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
    !pip install mecab-python3==0.7
    !pip install jaconv
    
    import MeCab
    wakati = MeCab.Tagger('-Owakati').parse
    yomi = MeCab.Tagger('-Oyomi').parse
    
else:
    from ccap.mecab_settings import yomi
    from ccap.mecab_settings import wakati

# 自作ライブラリ LAM の読み込み
if isColab:
    !git clone https://github.com/ShinAsakawa/ccap.git
    !git clone https://github.com/ShinAsakawa/lam.git

if isColab:
    # colab 上で termcolor の色制御が動作しないので，バージョンを下げる必要がある
    !pip install --upgrade termcolor==2.0.1 2>&1
    
    !pip install jupyter_contrib_nbextensions 2>&1 
    !jupyter nbextension enable codefolding/main 2>&1

## 1.2 パラメータ設定

語彙数を 10K 語から 20K 語に倍増しているのは，Fushimi1999 の語彙リストの未知語が存在したためである。

In [ ]:
%reload_ext autoreload
%autoreload 2

import torch
import lam
#device = lam.device  # CPU or GPU の選択
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from termcolor import colored

# シミュレーションに必要なパラメータの設定
params = {
    'traindata_size':  20000,   # 訓練データ数，NTT 日本語語彙特性の高頻度語を上位から抽出
    #'traindata_size': 301612,  # 訓練データ数，NTT 日本語語彙特性の高頻度語を上位から抽出
    'epochs': 30,               # 学習のためのエポック数
    'hidden_size': 64,          # 中間層のニューロン数
    #'hidden_size': 128,         # 中間層のニューロン数
    'random_seed': 42,          # 乱数の種。ダグラス・アダムス著「銀河ヒッチハイカーズガイド」

    # 以下 `source` と `target` を定義することで，別の課題を実行可能
    'source': 'orth',          # ['orth', 'phon', 'mora', 'mora_p', 'mora_p_r']
    'target': 'phon',          # ['orth', 'phon', 'mora', 'mora_p', 'mora_p_r']
    #'target': 'mora_p_r',     # ['orth', 'phon', 'mora', 'mora_p', 'mora_p_r']
    # 'orth': 書記素, 
    # 'phon': 音韻, 
    # 'mora': モーラ
    # 'mora_p': モーラを silius による音分解
    # 'mora_p_r': モーラの silius 音分解の逆
    'pretrained': False,          # True であれば訓練済ファイルを読み込む
    #'pretrained': True,          # True であれば訓練済ファイルを読み込む
    #'isTrain'   : True,          # True であれば学習する
    
    # 学習済のモデルパラメータを保存するファイル名
    'path_saved': '2023_0120lam_o2p_hid64_nttfreq20k.pt', 
    #'path_saved': '2022_0829lam_p2p_hid24_vocab10k.pt',
    #'path_saved': False,                      # 保存しない場合
    
    # 結果の散布図を保存するファイル名    
    'path_graph': '2023_0120lam_p2p_hid64_nttfreq20k.pdf',
    #'path_graph': False,                     # 保存しない場合

    'lr': 0.0001,                              # 学習率
    'dropout_p': 0.0,                         # ドロップアウト率
    'teacher_forcing_ratio': 0.5,             # 教師強制を行う確率
    'optim_func': torch.optim.Adam,           # 最適化アルゴリズム ['torch.optim.Adam', 'torch.optim.SGD', 'torch.optim.AdamW']
    'loss_func' :torch.nn.CrossEntropyLoss(), # 交差エントロピー損失 ['torch.nn.NLLLoss()', or 'torch.nn.CrossEntropyLoss()']
}

source = params['source']
target = params['target']
_src, _tgt = source+'_ids', target+'_ids'

## 1.3 Fushimi1999 データセット

In [ ]:
from termcolor import colored
import sys

verbose = False

fushimi1999 = {
    'HF___consist__': ['戦争', '倉庫', '医学', '注意', '記念', '番号', '料理', '完全', '開始', '印刷',
                       '連続', '予約', '多少', '教員', '当局', '材料', '夕刊', '労働', '運送', '電池' ], # consistent, 'high-frequency words
    'HF___inconsist': ['反対', '失敗', '作品', '指定', '実験', '決定', '独占', '独身', '固定', '食品',
                       '表明', '安定', '各種', '役所', '海岸', '決算', '地帯', '道路', '安打', '楽団' ], # inconsistent, 'high-frequency words
    'HF___atypical_': ['仲間', '夫婦', '人間', '神経', '相手', '反発', '化粧', '建物', '彼女', '毛糸', 
                       '場合', '台風', '夜間', '人形', '東西', '地元', '松原', '競馬', '大幅', '貸家' ], # inconsistent atypical, 'high-frequency words
    'LF___consist__': ['集計', '観察', '予告', '動脈', '理学', '信任', '任務', '返信', '医局', '低温', 
                       '区別', '永続', '持続', '試練', '満開', '軍備', '製材', '銀貨', '急送', '改選' ], # consistent, 'low-frequecy words
    'LF___inconsist': ['表紙', '指針', '熱帯', '作詞', '決着', '食費', '古代', '地形', '役場', '品種', 
                       '祝福', '金銭', '根底', '接種', '経由', '郷土', '街路', '宿直', '曲折', '越境' ], # inconsistent, 'low-frequency words
    'LF___atypical_': ['強引', '寿命', '豆腐', '出前', '歌声', '近道', '間口', '風物', '面影', '眼鏡', 
                       '居所', '献立', '小雨', '毛皮', '鳥居', '仲買', '頭取', '極上', '奉行', '夢路' ], # inconsistent atypical, 'low-frequncy words
    'HFNW_consist__': ['集学', '信別', '製信', '運学', '番送', '電続', '完意', '軍開', '動選', '当働', 
                       '予続', '倉理', '予少', '教池', '理任', '銀務', '連料', '開員', '注全', '記争' ], # consistent, 'high-character-frequency nonwords
    'HFNW_inconsist': ['作明', '風行', '失定', '指団', '決所', '各算', '海身', '東発', '楽験', '作代',
                       '反原', '独対', '歌上', '反定', '独定', '場家', '安種', '経着', '決土', '松合' ], # inconsistent biased, 'high-character-frequency nonwords
    'HFNW_ambiguous': ['表品', '実定', '人風', '神間', '相経', '人元', '小引', '指場', '毛所', '台手',
                       '間物', '道品', '出取', '建馬', '大婦', '地打', '化間', '面口', '金由', '彼間' ], # inconsistent ambigous, 'high-character-frequency nonwords
    'LFNW_consist__': ['急材', '戦刊', '返計', '印念', '低局', '労号', '満送', '永告', '試脈', '観備',
                       '材約', '夕局', '医庫', '任続', '医貨', '改練', '区温', '多始', '材刷', '持察' ], # consistent, 'low-character-frequency nonwords
    'LFNW_inconsist': ['食占', '表底', '宿帯', '決帯', '古費', '安敗', '役針', '近命', '眼道', '豆立',
                       '街直', '固路', '郷種', '品路', '曲銭', '献居', '奉買', '根境', '役岸', '祝折' ], # inconsistent biased, 'low-character-frequency nonwords
    'LFNW_ambiguous': ['食形', '接紙', '競物', '地詞', '強腐', '頭路', '毛西', '夜糸', '仲影', '熱福',
                       '寿前', '鳥雨', '地粧', '越種', '仲女', '極鏡', '夢皮', '居声', '貸形', '夫幅' ], # inconsistent ambigous, 'low-character-frequency nonwords
}


for k, v in fushimi1999.items():
    # 上のデータを表示
    print(colored(k, 'blue', attrs=['bold']), v)

fushimi1999_list = [] # 上のデータをリスト化
for k, v in fushimi1999.items():
    for _v in v:
        fushimi1999_list.append(_v)

if verbose:
    print(colored('# Fushimi1999 データから，訓練データに含まれているデータを表示する', 'blue', attrs=['bold']))
    for i, wrd in enumerate(fushimi1999_list):
        
        if wrd in train_wordlist:
            color = 'blue'
            idx = train_wordlist.index(wrd)
        else:
            color = 'red'
            idx = -1
        print(colored((f'{i:3d} wrd:{wrd},idx:{idx:5d}',
              f'orth_tkn2ids:{orth_tkn2ids(wrd)}', #o[_tgt]
                 ),color=color, attrs=['bold']))

print(f'fushimi1999_list:{fushimi1999_list}')        

In [ ]:
for i, wrd in enumerate(fushimi1999_list):
    _end = "\n" if (i+1) % 10 == 0 else ' '
    print(f'{i+1:3d}: {wrd}', end="") #
    for ch in wrd:
        print(f'({ch}:{_vocab.orth_vocab.index(ch):4d})', end="")
    if (i+1) % 5 == 0:
        print()
    else:
        print(' ', end="")

## 1.3 データセットの設定

In [ ]:
from tqdm.notebook import tqdm  #jupyter で実行時
import numpy as np
import os
import re
import gzip
import jaconv

class VOCAB():
    '''
    訓練データとしては，NTT 日本語語彙特性 (天野，近藤, 1999, 三省堂) の頻度データ，実際のファイル名としては `pslex71.txt` から頻度データを読み込んで，高頻度語を訓練データとする。
    ただし，検証データに含まれる単語は訓練データとして用いない。

    検証データとして，以下のいずれかを考える
    1. TLPA (藤田 他, 2000, 「失語症語彙検査」の開発，音声言語医学 42, 179-202)
    2. SALA 上智大学失語症語彙検査

    このオブジェクトクラスでは，
    `phon_vocab`, `orth_vocab`, `ntt_freq`, に加えて，単語の読みについて ntt_orth2hira によって読みを得ることにした。

    * `train_data`, `test_data` という辞書が本体である。
    各辞書の項目には，さらに
    `Vocab_ja.test_data[0].keys() = dict_keys(['orig', 'orth', 'phon', 'orth_ids', 'phon_ids', 'semem'])`

    各モダリティ共通トークンとして以下を設定した
    * <PAD>: 埋め草トークン
    * <EQW>: 単語終端トークン
    * <SOW>: 単語始端トークン
    * <UNK>: 未定義トークン

    このクラスで定義されるデータは 2 つの辞書である。すなわち 1. train_data, 2. tlpa_data である。
    各辞書は，次のような辞書項目を持つ。
    ```
    {0: {'orig': 'バス',
    'yomi': 'ばす',
    'orth': ['バ', 'ス'],
    'orth_ids': [695, 514],
    'orth_r': ['ス', 'バ'],
    'orth_ids_r': ['ス', 'バ'],
    'phon': ['b', 'a', 's', 'u'],
    'phon_ids': [23, 7, 19, 12],
    'phon_r': ['u', 's', 'a', 'b'],
    'phon_ids_r': [12, 19, 7, 23],
    'mora': ['ば', 'す'],
    'mora_r': ['す', 'ば'],
    'mora_ids': [87, 47],
    'mora_p': ['b', 'a', 's', 'u'],
    'mora_p_r': ['s', 'u', 'b', 'a'],
    'mora_p_ids': [6, 5, 31, 35],
    'mora_p_ids_r': [31, 35, 6, 5]},
    ```
    '''

    def __init__(self,
                 traindata_size = 10000,  # デフォルト語彙数
                 w2v=None,                # word2vec (gensim)
                 yomi=None,               # MeCab を用いた `読み` の取得のため`
                 ps71_fname:str=None,     # NTT 日本語語彙特性の頻度データファイル名
                 stop_list:list=[],       # ストップ単語リスト：訓練データから排除する単語リスト
                 #test_name='TLPA',  # or 'SALA',
                ):

        if yomi != None:
            self.yomi = yomi
        else:
            #from mecab_settings import yomi
            from ccap.mecab_settings import yomi
            self.yomi = yomi

        # 訓練語彙数の上限 `training_size` を設定
        self.traindata_size = traindata_size

        # `self.moraWakachi()` で用いる正規表現のあつまり 各条件を正規表現で表す
        self.c1 = '[うくすつぬふむゆるぐずづぶぷゔ][ぁぃぇぉ]' #ウ段＋「ァ/ィ/ェ/ォ」
        self.c2 = '[いきしちにひみりぎじぢびぴ][ゃゅぇょ]' #イ段（「イ」を除く）＋「ャ/ュ/ェ/ョ」
        self.c3 = '[てで][ぃゅ]' #「テ/デ」＋「ャ/ィ/ュ/ョ」
        self.c4 = '[ぁ-ゔー]' #カタカナ１文字（長音含む）
        self.c5 = '[ふ][ゅ]'
        ## self.c1 = '[ウクスツヌフムユルグズヅブプヴ][ァィェォ]' #ウ段＋「ァ/ィ/ェ/ォ」
        ## self.c2 = '[イキシチニヒミリギジヂビピ][ャュェョ]' #イ段（「イ」を除く）＋「ャ/ュ/ェ/ョ」
        ## self.c3 = '[テデ][ィュ]' #「テ/デ」＋「ャ/ィ/ュ/ョ」
        ## self.c4 = '[ァ-ヴー]' #カタカナ１文字（長音含む）
        ##cond = '('+c1+'|'+c2+'|'+c3+'|'+c4+')'
        self.cond = '('+self.c5+'|'+self.c1+'|'+self.c2+'|'+self.c3+'|'+self.c4+')'
        self.re_mora = re.compile(self.cond)
        ## 以上 `self.moraWakachi()` で用いる正規表現の定義

        self.orth_vocab, self.orth_freq = ['<PAD>', '<EOW>','<SOW>','<UNK>'], {}
        self.phon_vocab, self.phone_freq = ['<PAD>', '<EOW>','<SOW>','<UNK>'], {}
        self.phon_vocab = ['<PAD>', '<EOW>', '<SOW>', '<UNK>',\
                           'N', 'a', 'a:', 'e', 'e:', 'i', 'i:', 'i::', 'o', 'o:', 'o::', 'u', 'u:', \
                           'b', 'by', 'ch', 'd', 'dy', 'f', 'g', 'gy', 'h', 'hy', 'j', 'k', 'ky', \
                           'm', 'my', 'n', 'ny', 'p', 'py', 'q', 'r', 'ry', 's', 'sh', 't', 'ts', 'w', 'y', 'z']
        self.mora_vocab = ['<PAD>', '<EOW>', '<SOW>', '<UNK>',\
                           'ァ', 'ア', 'ィ', 'イ', 'ゥ', 'ウ', 'ェ', 'エ', 'ォ', 'オ', \
                           'カ', 'ガ', 'キ', 'ギ', 'ク', 'グ', 'ケ', 'ゲ', 'コ', 'ゴ', \
                           'サ', 'ザ', 'シ', 'ジ', 'ス', 'ズ', 'セ', 'ゼ', 'ソ', 'ゾ', \
                           'タ', 'ダ', 'チ', 'ヂ', 'ッ', 'ツ', 'ヅ', 'テ', 'デ', 'ト', 'ド', \
                           'ナ', 'ニ', 'ヌ', 'ネ', 'ノ', \
                           'ハ', 'バ', 'パ', 'ヒ', 'ビ', 'ピ', 'フ', 'ブ', 'プ', 'ヘ', 'ベ', 'ペ', 'ホ', 'ボ', 'ポ', \
                           'マ', 'ミ', 'ム', 'メ', 'モ', \
                           'ャ', 'ヤ', 'ュ', 'ユ', 'ョ', 'ヨ', \
                           'ラ', 'リ', 'ル', 'レ', 'ロ', 'ワ', 'ン', 'ー'] 
        
        # 全モーラリストを `mora_vocab` として登録
        self.mora_vocab=[
            '<PAD>', '<EOW>', '<SOW>', '<UNK>',
            'ぁ', 'あ', 'ぃ', 'い', 'ぅ', 'う', 'うぃ', 'うぇ', 'うぉ', 'ぇ', 'え', 'お',
            'か', 'が', 'き', 'きゃ', 'きゅ', 'きょ', 'ぎ', 'ぎゃ', 'ぎゅ', 'ぎょ', 'く', 'くぁ', 'くぉ', 'ぐ', 'ぐぁ', 'け', 'げ', 'こ', 'ご',
            'さ', 'ざ', 'し', 'しぇ', 'しゃ', 'しゅ', 'しょ', 'じ', 'じぇ', 'じゃ', 'じゅ', 'じょ', 'す', 'ず', 'せ', 'ぜ', 'そ', 'ぞ',
            'た', 'だ', 'ち', 'ちぇ', 'ちゃ', 'ちゅ', 'ちょ', 'ぢ', 'ぢゃ', 'ぢょ', 'っ', 'つ', 'つぁ', 'つぃ', 'つぇ', 'つぉ', 'づ', 'て',
            'てぃ', 'で', 'でぃ', 'でゅ', 'と', 'ど',
            'な', 'に', 'にぇ', 'にゃ', 'にゅ', 'にょ', 'ぬ', 'ね', 'の',
            'は', 'ば', 'ぱ', 'ひ', 'ひゃ', 'ひゅ', 'ひょ', 'び', 'びゃ', 'びゅ', 'びょ', 'ぴ', 'ぴゃ', 'ぴゅ', 'ぴょ',
            'ふ', 'ふぁ', 'ふぃ', 'ふぇ', 'ふぉ', 'ふゅ', 'ぶ', 'ぷ', 'へ', 'べ', 'ぺ', 'ほ', 'ぼ', 'ぽ',
            'ま', 'み', 'みゃ', 'みゅ', 'みょ', 'む', 'め', 'も',
            'や', 'ゆ', 'よ', 'ら', 'り', 'りゃ', 'りゅ', 'りょ', 'る', 'れ', 'ろ', 'ゎ', 'わ', 'ゐ', 'ゑ', 'を', 'ん', 'ー',
            # 2022_1017 added
            'ずぃ', 'ぶぇ', 'ぶぃ', 'ぶぁ', 'ゅ', 'ぶぉ', 'いぇ', 'ぉ', 'くぃ', 'ひぇ', 'くぇ', 'ぢゅ', 'りぇ',
        ]
        
        # モーラに用いる音を表すリストを `mora_p_vocab` として登録
        self.mora_p_vocab = ['<PAD>', '<EOW>', '<SOW>', '<UNK>',  \
        'N', 'a', 'b', 'by', 'ch', 'd', 'dy', 'e', 'f', 'g', 'gy', 'h', 'hy', 'i', 'j', 'k', 'ky', \
        'm', 'my', 'n', 'ny', 'o', 'p', 'py', 'q', 'r', 'ry', 's', 'sh', 't', 'ts', 'u', 'w', 'y', 'z']

        # 母音を表す音から ひらがな への変換表を表す辞書を `vow2hira` として登録
        self.vow2hira = {'a':'あ', 'i':'い', 'u':'う', 'e':'え', 'o':'お', 'N':'ん'}

        self.mora_freq = {'<PAD>':0, '<EOW>':0, '<SOW>':0, '<UNK>':0}
        self.mora_p = {}

        # NTT 日本語語彙特性データから，`self.train_data` を作成
        self.ntt_freq, self.ntt_orth2hira = self.make_ntt_freq_data(ps71_fname=ps71_fname)
        self.ntt_freq_vocab = self.set_train_vocab()
        self.train_data, self.excluded_data = {}, []
        max_orth_length, max_phon_length, max_mora_length, max_mora_p_length = 0, 0, 0, 0
        self.train_vocab = []
        
        num = '０１２３４５６７８９'
        alpha = 'ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ'   # ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ'
        hira = 'あいうえおかがきぎくぐけげこごさざしじすずせぜそぞただちぢつづてでとどなにぬねのはばぱひびぴふぶぷへべぺほぼぽまみむめもやゆよらりるれろわゐゑをん'
        kata = 'アイウエオカガキギクグケゲコゴサザシジスズセゼソゾタダチヂツヅテデトドナニヌネノハバパヒビピフブプヘベペホボポマミムメモヤユヨラリルレロワヰヱヲン'
        onechars = hira+alpha+num # +kata
        for i, orth in enumerate(onechars):
            
            if not orth in self.train_vocab:
                self.train_vocab.append(orth)
            _yomi = yomi(orth).strip()
            hira = jaconv.kata2hira(_yomi)
            phon_juli = jaconv.hiragana2julius(hira).split(' ')
                
            # 書記素 ID リスト `orth_ids` に書記素を登録
            for o in orth:
                if not o in self.orth_vocab:
                    self.orth_vocab.append(o)
            orth_ids = [self.orth_vocab.index(o) for o in orth]
            phon_ids = [self.phon_vocab.index(p) if p in self.phon_vocab else self.phon_vocab.index('<UNK>') for p in phon_juli]
            
            self.train_data[i] = {
                'orig':orth,
                'orth':orth,
                'yomi':_yomi,
                'phon':phon_juli,
                'phon_ids': phon_ids,
                'orth_ids': orth_ids
            }


        for orth in tqdm(self.ntt_freq_vocab):
            if orth in stop_list:       # stop list に登録されていたらスキップ
                continue
                
            if orth in self.train_vocab: # すでに登録されている単語であればスキップ
                continue
            else:
                self.train_vocab.append(orth)
                
            n_i = len(self.train_data)

            # 書記素 `orth` から 読みリスト，音韻表現リスト，音韻表現反転リスト，
            # 書記表現リスト，書記表現反転リスト，モーラ表現リスト，モーラ表現反転リスト の 7 つのリストを得る
            _yomi, _phon, _phon_r, _orth, _orth_r, _mora, _mora_r = self.get7lists_from_orth(orth_wrd=orth)

            # 音韻語彙リスト `self.phon_vocab` に音韻が存在していれば True そうでなければ False というリストを作成し，
            # そのリスト無いに False があれば，排除リスト `self.excluded_data` に登録する
            #if False in [True if p in self.phon_vocab else False for p in _phon]:
            #    self.excluded_data.append(orth)
            #    continue

            phon_ids, phon_ids_r, orth_ids, orth_ids_r, mora_ids, mora_ids_r = self.get6ids(_phon, _orth, _yomi)
            _yomi, _mora1, _mora1_r, _mora, _mora_ids, _mora_p, _mora_p_r, _mora_p_ids, _mora_p_ids_r, _juls = self.yomi2mora_transform(_yomi)
            self.train_data[n_i] = {'orig': orth, 'yomi': _yomi,
                                    'orth':_orth, 'orth_ids': orth_ids, 'orth_r': _orth_r, 'orth_ids_r': orth_ids_r,
                                    'phon':_phon, 'phon_ids': phon_ids, 'phon_r': _phon_r, 'phon_ids_r': phon_ids_r,
                                    'mora': _mora1, 'mora_r': _mora1_r, 'mora_ids': _mora_ids, 'mora_p': _mora_p,
                                    'mora_p_r': _mora_p_r, 'mora_p_ids': _mora_p_ids, 'mora_p_ids_r': _mora_p_ids_r,
                                   }
            len_orth, len_phon, len_mora, len_mora_p = len(_orth), len(_phon), len(_mora), len(_mora_p)
            max_orth_length = len_orth if len_orth > max_orth_length else max_orth_length
            max_phon_length = len_phon if len_phon > max_phon_length else max_phon_length
            max_mora_length = len_mora if len_mora > max_mora_length else max_mora_length
            max_mora_p_length = len_mora_p if len_mora_p > max_mora_p_length else max_mora_p_length
            
            if len(self.train_data) >= self.traindata_size: # 上限値に達したら終了する
                #self.train_vocab = [self.train_data[x]['orig'] for x in self.train_data.keys()]
                break

        self.max_orth_length = max_orth_length
        self.max_phon_length = max_phon_length
        self.max_mora_length = max_mora_length
        self.max_mora_p_length = max_mora_p_length
        


    def yomi2mora_transform(self, yomi):
        """ひらがな表記された引数 `yomi` から，日本語の 拍(モーラ)  関係のデータを作成する
        引数:
        yomi:str ひらがな表記された単語 UTF-8 で符号化されていることを仮定している

        戻り値:
        yomi:str 入力された引数
        _mora1:list[str] `_mora` に含まれる長音 `ー` を直前の母音で置き換えた，モーラ単位の分かち書きされた文字列のリスト
        _mora1_r:list[str] `_mora1` を反転させた文字列リスト
        _mora:list[str] `self.moraWakatchi()` によってモーラ単位で分かち書きされた文字列のリスト
        _mora_ids:list[int] `_mora` を対応するモーラ ID で置き換えた整数値からなるリスト
        _mora_p:list[str] `_mora` を silius によって音に変換した文字列リスト
        _mora_p_r:list[str] `_mora_p` の反転リスト
        _mora_p_ids:list[int] `mora_p` の各要素を対応する 音 ID に変換した数値からなるリスト
        _mora_p_ids_r:list[int] `mora_p_ids` の各音を反転させた数値からなるリスト
        _juls:list[str]: `yomi` を julius 変換した音素からなるリスト
        """
        _mora = self.moraWakachi(yomi) # 一旦モーラ単位の分かち書きを実行して `_mora` に格納

        # 単語をモーラ反転した場合に長音「ー」の音が問題となるので，長音「ー」を母音で置き換えるためのプレースホルダとして. `_mora` を用いる
        _mora1 = _mora.copy()

        # その他のプレースホルダの初期化，モーラ，モーラ毎 ID, モーラ音素，モーラの音素の ID， モーラ音素の反転，モーラ音素の反転 ID リスト
        mora_ids, mora_p, mora_p_ids, mora_p_r, _mora_p_ids_r = [], [], [], [], []
        _m0 = 'ー' # 長音記号

        for i, _m in enumerate(_mora): # 各モーラ単位の処理と登録

            __m = _m0 if _m == 'ー' else _m               # 長音だったら，前音の母音を __m とし，それ以外は自分自身を __m に代入
            _mora1[i] = __m                               # 長音を変換した結果を格納
            mora_ids.append(self.mora_vocab.index(__m))  # モーラを ID 番号に変換
            mora_p += jaconv.hiragana2julius(__m).split()
            #_mora_p += self.mora2jul[__m]                 # モーラを音素に変換して `_mora_p` に格納

            # 変換した音素を音素 ID に変換して，`_mora_p_ids` に格納
            #for _p in jaconv.hiragana2julius(_m).split():
            #    idx = self.phon_vocab.index(_p)
            #    mora_p_ids.append(idx)
            #mora_p_ids = [self.phon_vocab.index(_p) for _p in jaconv.hiragana2julius(__m).split()]
            #_mora_p_ids += [self.mora_p_vocab.index(_p) for _p in self.mora2jul[__m]]

            if not _m in self.mora_freq: # モーラの頻度表を集計
                self.mora_freq[__m] = 1
            else:
                self.mora_freq[__m] +=1

            if self.hira2julius(__m)[-1] in self.vow2hira:      # 直前のモーラの最終音素が母音であれば
                _m0 = self.vow2hira[self.hira2julius(__m)[-1]]  # 直前の母音を代入しておく。この処理が 2022_0311 でのポイントであった
                
        mora_p_ids = [self.phon_vocab.index(_p) for _p in mora_p]

        # モーラ分かち書きした単語 _mora1 の反転を作成し `_mora1_r` に格納
        _mora1_r = [m for m in _mora1[::-1]]
        mora_p_r = []
        for _m in _mora1_r:                   # 反転した各モーラについて
            # モーラ単位で julius 変換して音素とし `_mora_p_r` に格納
            for _jul in jaconv.hiragana2julius(_m).split():
                mora_p_r.append(_jul)
            #_mora_p_r += self.mora2jul[_m]

            # mora_p_r に格納した音素を音素 ID に変換し mora_p_ids に格納
            #mora_p_ids += [self.mora_p_vocab.index(_p) for _p in self.mora2jul[_m]]
            
        mora_p_ids_r = [self.phon_vocab.index(_m) for _m in mora_p_r]
        _juls = self.hira2julius(yomi)

        return yomi, _mora1, _mora1_r, _mora, mora_ids, mora_p, mora_p_r, mora_p_ids, mora_p_ids_r, _juls

    def orth2orth_ids(self, 
                      orth:str):
        orth_ids = [self.orth_vocab.index(ch) if ch in self.orth_vocab else self.orth_vocab.index('<UNK>') for ch in orth]
        return orth_ids

    def phon2phon_ids(self, 
                      phon:list):
        phon_ids = [self.phon_vocab.index(ph) if ph in self.phon_vocab else self.phon_vocab.index('<UNK>') for ph in phon]
        return phon_ids
    
    def yomi2phon_ids(self,
                      yomi:str):
        phon_ids = []
        for _jul in self.hira2julius(yomi):
            if _jul in self.phon_vocab:
                ph = self.phon_vocab.index(_jul)
            else:
                ph = self.phon_vocab.index('<UNK>')
            phon_ids.append(ph)
        return phon_ids
    
    
    def orth_ids2tkn(self, ids:list):
        return [self.orth_vocab[idx] for idx in ids]

    def orth_tkn2ids(self, tkn:list):
        return [self.orth_vocab.index(_tkn) if _tkn in self.orth_vocab else self.orth_vocab.index('<UNK>') for _tkn in tkn]

    def mora_p_ids2tkn(self, ids:list):
        return [self.mora_p_vocab[idx] for idx in ids]

    def mora_p_tkn2ids(self, tkn:list):
        return [self.mora_p_vocab.index(_tkn) if _tkn in self.mora_p_vocab else self.mora_p_vocab('<UNK>') for _tkn in tkn]

    def mora_ids2tkn(self, ids:list):
        return [self.mora_vocab[idx] for idx in ids]

    def mora_tkn2ids(self, tkn:list):
        return [self.mora_vocab.index(_tkn) if _tkn in self.mora_vocab else self.mora_vocab('<UNK>') for _tkn in tkn]

    def phon_ids2tkn(self, ids:list):
        return [self.phon_vocab[idx] for idx in ids]

    def phon_tkn2ids(self, tkn:list):
        return [self.phon_vocab.index(_tkn) if _tkn in self.phon_vocab else self.phon_vocab.index('<UNK>') for _tkn in tkn]

    def get6ids(self, _phon, _orth, yomi):

        # 音韻 ID リスト `phon_ids` に音素を登録する
        phon_ids = [self.phon_vocab.index(p) if p in self.phon_vocab else self.phon_vocab.index('<UNK>') for p in _phon]

        # 直上の音韻 ID リストの逆転を作成
        phon_ids_r = [p_id for p_id in phon_ids[::-1]]

        # 書記素 ID リスト `orth_ids` に書記素を登録
        for o in _orth:
            if not o in self.orth_vocab:
                self.orth_vocab.append(o)
        orth_ids = [self.orth_vocab.index(o) for o in _orth]

        # 直上の書記素 ID リストの逆転を作成
        orth_ids_r = [o_id for o_id in orth_ids[::-1]]
        #orth_ids_r = [o_id for o_id in _orth[::-1]]

        mora_ids = []
        for _p in self.hira2julius(yomi):
            mora_ids.append(self.phon_vocab.index(_p) if _p in self.phon_vocab else self.phon_vocab.index('<UNK>'))

        mora_ids_r = [m_id for m_id in mora_ids]
        return phon_ids, phon_ids_r, orth_ids, orth_ids_r, mora_ids, mora_ids_r


    def moraWakachi(self, hira_text):
        """ ひらがなをモーラ単位で分かち書きする
        https://qiita.com/shimajiroxyz/items/a133d990df2bc3affc12"""

        return self.re_mora.findall(hira_text)


    def _kana_moraWakachi(kan_text):
        self.cond = '('+self.c1+'|'+self.c2+'|'+self.c3+'|'+self.c4+')'
        self.re_mora = re.compile(self.cond)

        return re_mora.findall(kana_text)

    
    def get7lists_from_orth(self, orth_wrd):
        """書記素 `orth` から 読みリスト，音韻表現リスト，音韻表現反転リスト，
        書記表現リスト，書記表現反転リスト，モーラ表現リスト，モーラ表現反転リスト の 7 つのリストを得る"""

        # 単語の表層形を，読みに変換して `_yomi` に格納
        # ntt_orth2hira という命名はおかしかったから修正 2022_0309
        if orth_wrd in self.ntt_orth2hira:
            _yomi = self.ntt_orth2hira[orth_wrd]
        else:
            _yomi = jaconv.kata2hira(self.yomi(orth_wrd).strip())

        # `_yomi` を julius 表記に変換して `_phon` に代入
        _phon = self.hira2julius(_yomi)# .split(' ')

        # 直上の `_phon` の逆転を作成して `_phone_r` に代入
        _phon_r = [_p_id for _p_id in _phon[::-1]]

        # 書記素をリストに変換
        _orth = [c for c in orth_wrd]

        # 直上の `_orth` の逆転を作成して `_orth_r` に代入
        _orth_r = [c for c in _orth[::-1]]

        #_mora = self.moraWakachi(jaconv.hira2kata(_yomi))
        _mora = self.moraWakachi(_yomi)
        for _m in _mora:
            if not _m in self.mora_vocab:
                self.mora_vocab.append(_m)
            for _j in self.hira2julius(_m):
                if not _j in self.mora_p:
                    self.mora_p[_j] = 1
                else:
                    self.mora_p[_j] += 1
        _mora_r = [_m for _m in _mora[::-1]]
        return _yomi, _phon, _phon_r, _orth, _orth_r, _mora, _mora_r


    def hira2julius(self, text:str)->str:
        """`jaconv.hiragana2julius()` では未対応の表記を扱う"""
        text = text.replace('ゔぁ', ' b a')
        text = text.replace('ゔぃ', ' b i')
        text = text.replace('ゔぇ', ' b e')
        text = text.replace('ゔぉ', ' b o')
        text = text.replace('ゔゅ', ' by u')

        #text = text.replace('ぅ゛', ' b u')
        text = jaconv.hiragana2julius(text).split()
        return text


    def __len__(self)->int:
        return len(self.train_data)

    
    def __call__(self, x:int)->dict:
        return self.train_data[x]

    
    def __getitem__(self, x:int)->dict:
        return self.train_data[x]

    
    def set_train_vocab(self):
    #def set_train_vocab_minus_test_vocab(self):
        """JISX2008-1990 コードから記号とみなしうるコードを集めて ja_symbols とする
        記号だけから構成されている word2vec の項目は排除するため
        """
        self.ja_symbols = '、。，．・：；？！゛゜´\' #+ \'｀¨＾‾＿ヽヾゝゞ〃仝々〆〇ー—‐／＼〜‖｜…‥‘’“”（）〔〕［］｛｝〈〉《》「」『』【】＋−±×÷＝≠＜＞≦≧∞∴♂♀°′″℃¥＄¢£％＃＆＊＠§☆★○●◎◇◆□■△▲▽▼※〒→←↑↓〓∈∋⊆⊇⊂⊃∪∩∧∨¬⇒⇔∀∃∠⊥⌒∂∇≡≒≪≫√∽∝∵∫∬Å‰♯♭♪†‡¶◯#ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ'
        #self.ja_symbols_normalized = jaconv.normalize(self.ja_symbols)

        print(f'# 訓練に用いる単語の選定 {self.traindata_size} 語')
        vocab = []; i=0
        while i<len(self.ntt_freq):
            word = self.ntt_freq[i]
            if word == '\u3000': # NTT 日本語の語彙特性で，これだけ変なので特別扱い
                i += 1
                continue

            # 良い回避策が見つからないので，以下の行の変換だけ特別扱いしている
            word = jaconv.normalize(word).replace('・','').replace('ヴ','ブ')

            if (not word in self.ja_symbols) and (not word.isascii()): # and (word in self.w2v):
                
                if not word in vocab:
                    vocab.append(word)
                    if len(vocab) >= self.traindata_size:
                        return vocab
            i += 1
        return vocab


    def make_ntt_freq_data(self,
                           ps71_fname:str=None):

        print('# NTT日本語語彙特性 (天野，近藤; 1999, 三省堂)より頻度情報を取得')

        if ps71_fname == None:
            #データファイルの保存してあるディレクトリの指定
            ntt_dir = 'ccap'
            psy71_fname = 'psylex71utf8.txt'  # ファイル名
            psy71_fname = 'psylex71utf8.txt.gz'  # ファイル名
            #with gzip.open(os.path.join(ntt_dir,psy71_fname), 'r') as f:
            with gzip.open(os.path.join(ntt_dir,psy71_fname), 'rt', encoding='utf-8') as f:
                ntt71raw = f.readlines()
        else:
            with open(ps71_fname, 'r') as f:
                ntt71raw = f.readlines()

        tmp = [line.split(' ')[:6] for line in ntt71raw]
        tmp2 = [[int(line[0]),line[2],line[4],int(line[5]), line[3]] for line in tmp]
        #単語ID(0), 単語，品詞，頻度 だけ取り出す

        ntt_freq = {x[0]-1:{'単語':jaconv.normalize(x[1]),
                            '品詞':x[2],
                            '頻度':x[3],
                            'よみ':jaconv.kata2hira(jaconv.normalize(x[4]))
                            } for x in tmp2}
        #ntt_freq = {x[0]-1:{'単語':x[1],'品詞':x[2],'頻度':x[3], 'よみ':x[4]} for x in tmp2}
        ntt_orth2hira = {ntt_freq[x]['単語']:ntt_freq[x]['よみ'] for x in ntt_freq}
        #print(f'#登録総単語数: {len(ntt_freq)}')

        Freq = np.zeros((len(ntt_freq)), dtype=np.uint)  #ソートに使用する numpy 配列
        for i, x in enumerate(ntt_freq):
            Freq[i] = ntt_freq[i]['頻度']

        Freq_sorted = np.argsort(Freq)[::-1]  #頻度降順に並べ替え

        # self.ntt_freq には頻度順に単語が並んでいる
        return [ntt_freq[x]['単語']for x in Freq_sorted], ntt_orth2hira

_vocab = VOCAB(
    traindata_size=params['traindata_size'],     
    w2v=None, 
    yomi=yomi,
    stop_list=fushimi1999_list) 

train_wordlist = [v['orig'] for k, v in _vocab.train_data.items()]

top_n = 300
print(f'語彙先頭の項目 {top_n} を印字')
for i, wrd in enumerate(train_wordlist[:top_n]):
    _end = " " if (i+1) % 10 != 0 else "\n"
    print((i+1, wrd), end=_end)

In [ ]:
class Train_dataset(torch.utils.data.Dataset):

    def __init__(self,
                 data:VOCAB=None,
                 source_vocab:list=None,
                 target_vocab:list=None,
                 source_ids:str=None,
                 target_ids:str=None,
                ):

        if data == None:
            self.data = VOCAB()
        else:
            self.data = data
        self.order = {i:self.data[x] for i, x in enumerate(self.data)}

        self.source_ids = source_ids
        self.target_ids = target_ids
        self.source_vocab = source_vocab
        self.target_vocab = target_vocab

    def __len__(self)->int:
        return len(self.data)

    def __getitem__(self, x:int):
        return self.order[x][self.source_ids] + [self.source_vocab.index('<EOW>')], self.order[x][self.target_ids] + [self.target_vocab.index('<EOW>')]

    def convert_source_ids_to_tokens(self, ids:list):
        return [self.source_vocab[idx] for idx in ids]

    def convert_target_ids_to_tokens(self, ids:list):
        return [self.target_vocab[idx] for idx in ids]


class Val_dataset(torch.utils.data.Dataset):
    """同じく検証データセットの定義"""

    def __init__(self,
                data:dict=None,
                source_vocab:list=None,
                target_vocab:list=None,
                source_ids:str=None,
                target_ids:str=None,
                ):

        if 'pdata' in str(data.keys()):
            self.data = data['pdata']
        else:
            self.data = data

        self.order = {i:self.data[x] for i, x in enumerate(self.data)}

        self.target_ids = target_ids
        self.source_ids = source_ids

        self.source_vocab = source_vocab if source_vocab != None else VOCAB().mora_p_vocab
        self.target_vocab = target_vocab if target_vocab != None else VOCAB().mora_p_vocab


    def __len__(self)->int:
        return len(self.data)

    def __getitem__(self, x:int):
        return self.order[x][self.source_ids] + [self.source_vocab.index('<EOW>')], self.order[x][self.target_ids] + [self.target_vocab.index('<EOW>')]

    def convert_source_ids_to_tokens(self, ids:list):
        return [self.source_vocab[idx] for idx in ids]

    def convert_target_ids_to_tokens(self, ids:list):
        return [self.target_vocab[idx] for idx in ids]

def make_X_vals(_dataset=None,
                source_vocab=None,
                target_vocab=None,
                source_ids=None,
                target_ids=None,
                ):

    if _dataset == None:
        print('_dataset must be set')
        sys.exit()

    sala_r29val = Val_dataset(
        data=_dataset['sala_r29'],
        source_vocab=source_vocab,
        target_vocab=target_vocab,
        source_ids=source_ids,
        target_ids=target_ids)

    sala_r30val = Val_dataset(
        data=_dataset['sala_r30'],
        source_vocab=source_vocab,
        target_vocab=target_vocab,
        source_ids=source_ids,
        target_ids=target_ids)

    sala_r31val = Val_dataset(
        data=_dataset['sala_r31'],
        source_vocab=source_vocab,
        target_vocab=target_vocab,
        source_ids=source_ids,
        target_ids=target_ids)

    tlpa2val    = Val_dataset(
        data=_dataset['tlpa2'],
        source_vocab=source_vocab,
        target_vocab=target_vocab,
        source_ids=source_ids,
        target_ids=target_ids)

    tlpa3val    = Val_dataset(
        data=_dataset['tlpa3'],
        source_vocab=source_vocab,
        target_vocab=target_vocab,
        source_ids=source_ids,
        target_ids=target_ids)

    tlpa4val    = Val_dataset(
        data=_dataset['tlpa4'],
        source_vocab=source_vocab,
        target_vocab=target_vocab,
        source_ids=source_ids,
        target_ids=target_ids)

    X_vals = { 
        'sala_r29val': sala_r29val,
        'sala_r30val': sala_r30val,
        'sala_r31val': sala_r31val,
        'tlpa2val': tlpa2val, 
        'tlpa3val': tlpa3val, 
        'tlpa4val': tlpa4val}

    return X_vals


def make_vocab_dataset(_dict:dict, vocab:VOCAB=None)->dict:
    """上記 VOCAB を用いた下請け関数
    読み，音韻，モーラなどの情報を作成してデータセットといしての体裁を整える"""
    
    _data = {}
    if vocab == None:
        vocab = VOCAB()
    x = [x[0] for x in _dict.values()]
    for _x in x:
        i = len(_data)  # 連番の番号を得る
        orth = vocab.ntt_orth2hira[_x] if _x in vocab.ntt_orth2hira else _x
        _yomi, _phon, _phon_r, _orth, _orth_r, _mora, _mora_r = vocab.get7lists_from_orth(orth)
        phon_ids, phon_ids_r, orth_ids, orth_ids_r, mora_ids, mora_ids_r = vocab.get6ids(_phon, _orth, _yomi)
        _yomi, _mora1, _mora1_r, _mora, _mora_ids, _mora_p, _mora_p_r, _mora_p_ids, _mora_p_ids_r, _juls = vocab.yomi2mora_transform(_yomi)
        _data[i] = {'orig': orth, 
                    'yomi': _yomi, 
                    'orth':_orth, 'orth_ids': orth_ids, 'orth_r': _orth_r, 'orth_ids_r': orth_ids_r,
                    'phon':_phon, 'phon_ids': phon_ids, 'phon_r': _phon_r, 'phon_ids_r': phon_ids_r,
                    'mora': _mora1, 'mora_r': _mora1_r, 'mora_ids': _mora_ids, 'mora_p': _mora_p,
                    'mora_p_r': _mora_p_r, 'mora_p_ids': _mora_p_ids, 'mora_p_ids_r': _mora_p_ids_r, }
    return _data    

In [ ]:
def get_soure_and_target_from_params(
    params=None,
    _vocab=None,
    source=None,
    target=None,
    is_print:bool=True):

    if source == 'orth':
        source_vocab = _vocab.orth_vocab
        source_ids = 'orth_ids'
    elif source == 'phon':
        source_vocab = _vocab.phon_vocab
        source_ids = 'phon_ids'
    elif source == 'mora':
        source_vocab = _vocab.mora_vocab
        source_ids = 'mora_ids'
    elif source == 'mora_p':
        source_vocab = _vocab.mora_p_vocab
        source_ids = 'mora_p_ids'
    elif source == 'mora_p_r':
        source_vocab = _vocab.mora_p_vocab
        source_ids = 'mora_p_ids_r'

    if target == 'orth':
        target_vocab = _vocab.orth_vocab
        target_ids = 'orth_ids'
    elif target == 'phon':
        target_vocab = _vocab.phon_vocab
        target_ids = 'phon_ids'
    elif target == 'mora':
        target_vocab = _vocab.mora_vocab
        target_ids = 'mora_ids'
    elif target == 'mora_p':
        target_vocab = _vocab.mora_p_vocab
        target_ids = 'mora_p_ids'
    elif target == 'mora_p_r':
        target_vocab = _vocab.mora_p_vocab
        target_ids = 'mora_p_ids_r'

    if is_print:
        print(colored(f'source:{source}','blue', attrs=['bold']), f'{source_vocab}')
        print(colored(f'target:{target}','cyan', attrs=['bold']), f'{target_vocab}')
        print(colored(f'source_ids:{source_ids}','blue', attrs=['bold']), f'{source_ids}')
        print(colored(f'target_ids:{target_ids}','cyan', attrs=['bold']), f'{target_ids}')

    return source_vocab, source_ids, target_vocab, target_ids

In [ ]:
# _max_len はアテンション機構のデコーダで必要になるため，全条件で最長の長さを指定する必要がある
_max_len = _vocab.max_orth_length
_max_len = _max_len if _max_len > _vocab.max_phon_length else _vocab.max_phon_length
_max_len = _max_len if _max_len > _vocab.max_mora_length else _vocab.max_mora_length
_max_len = _max_len if _max_len > _vocab.max_mora_p_length else _vocab.max_mora_p_length
_vocab.max_length = _max_len + 1
print(colored(f'_vocab.max_length: {_vocab.max_length}', 'blue', attrs=['bold']))

# ソース，すなわち encoder 側の，項目番号，項目 ID，decoder 側の項目，項目 ID を設定
source_vocab, source_ids, target_vocab, target_ids = get_soure_and_target_from_params(
    #params=None,
    _vocab=_vocab,
    source=source,
    target=target,
    is_print=False)
    #is_print=True)

print(colored(f'source:{source}','blue', attrs=['bold']), f'{sorted(source_vocab)}')
print(colored(f'target:{target}','cyan', attrs=['bold']), f'{sorted(target_vocab)}')
#print(colored(f'source_ids:{source_ids}','blue', attrs=['bold']), f'{sorted(source_ids)}')
#print(colored(f'target_ids:{target_ids}','cyan', attrs=['bold']), f'{sorted(target_ids)}')


# 検証データとして，TLPA と SALA のデータを用いる
tlpa1, tlpa2, tlpa3, tlpa4, sala_r29, sala_r30, sala_r31 = lam.read_json_tlpa1234_sala_r29_30_31(
    json_fname='lam/2022_0508SALA_TLPA.json')

_dataset = {}
_data_names = ['tlpa2', 'tlpa3', 'tlpa4', 'sala_r29', 'sala_r30', 'sala_r31']
for data in _data_names:
    _dataset[data] = {'rawdata':eval(data),
                      'pdata': make_vocab_dataset(eval(data),vocab=_vocab)}

# 以下は後から付け足したので，コードが汚くなっている。
# 時間ができたらコードの整理をすること
X_vals = make_X_vals(_dataset=_dataset,
                         source_vocab=source_vocab,
                         target_vocab=target_vocab,
                         source_ids=source_ids,
                         target_ids=target_ids)

### 1.3.1 任意の単語 orthography を変換するための関数

### 1.3.2 SALA and TLPA dataset

In [ ]:
# 検証データとして，TLPA と SALA のデータを用いる
tlpa1, tlpa2, tlpa3, tlpa4, sala_r29, sala_r30, sala_r31 = lam.read_json_tlpa1234_sala_r29_30_31(
    json_fname='lam/2022_0508SALA_TLPA.json')

_dataset = {}
_data_names = ['tlpa2', 'tlpa3', 'tlpa4', 'sala_r29', 'sala_r30', 'sala_r31']
for data in _data_names:
    _dataset[data] = {'rawdata':eval(data),
                      'pdata': make_vocab_dataset(eval(data), vocab=_vocab)}

# 以下は後から付け足したので，コードが汚くなっている。
# 時間ができたらコードの整理をすること
X_vals = make_X_vals(_dataset=_dataset,
                         source_vocab=source_vocab,
                         target_vocab=target_vocab,
                         source_ids=source_ids,
                         target_ids=target_ids)

_data_names = ['tlpa2', 'tlpa3', 'tlpa4', 'sala_r29', 'sala_r30', 'sala_r31']    
for data in _data_names:
    print(colored(data, 'blue', attrs=['bold']), eval(data))

In [ ]:
import os
verbose = True
_src, _tgt = source+'_ids', target+'_ids'

if verbose:
    for i, w in enumerate(fushimi1999_list):
        ids = _vocab.orth_tkn2ids(w)
        tkn = _vocab.orth_ids2tkn(ids)
        print(i, w, tkn, ids)
        #os.system(f"say {w} --voice kyoko")

### 1.3.3 一文字データセットの定義

In [ ]:
class Onechar_dataset(torch.utils.data.Dataset): #_vocab.train_data):
    def __init__(self,
                 source:str='orth',
                 target:str='mora_p',
                 source_vocab=source_vocab,
                 target_vocab=target_vocab,
                ):
        super().__init__()

        _src = source
        _tgt = target

        _src = 'mora' if _src == 'mora_p' else _src
        _tgt = 'mora' if _tgt == 'mora_p' else _tgt
        _src, _tgt = _src+'_ids', _tgt+'_ids'
        
        digit_alpha = '０１２３４５６７８９ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ' # ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ'
        hira = 'あいうえおかがきぎくぐけげこごさざしじすずせぜそぞただちぢつづてでとどなにぬねのはばぱひびぴふぶぷへべぺほぼぽまみむめもやゆよらりるれろわゐゑをん'
        kata = 'アイウエオカガキギクグケゲコゴサザシジスズセゼソゾタダチヂツヅテデトドナニヌネノハバパヒビピフブプヘベペホボポマミムメモヤユヨラリルレロワヰヱヲン'
        onechars = digit_alpha+hira #+kata
        
        data_dict = {}
        for i, orth in enumerate(onechars):
            _yomi = yomi(orth).strip()
            hira = jaconv.kata2hira(_yomi)
            phon_juli = jaconv.hiragana2julius(hira).split(' ')
            phon_ids = _vocab.phon_tkn2ids(phon_juli)
            orth_ids = _vocab.orth_tkn2ids(orth)
            
            _data = {'_yomi':_yomi,
                     'phon':phon_juli,
                     'phon_ids':phon_ids,
                     'orth':orth,
                     'orth_ids':orth_ids}
            __src, __tgt = _data[_src], _data[_tgt]
            data_dict[i] = {'yomi':_yomi,
                            'orth':orth,
                            'src':__src,
                            'tgt':__tgt,
                            '_phon':phon_juli,
                            'phon_ids':phon_ids,
                            'orth_ids':orth_ids}

        self.source_vocab = source_vocab if source_vocab != None else VOCAB().mora_p_vocab
        self.target_vocab = target_vocab if target_vocab != None else VOCAB().mora_p_vocab
            
            
        self.data_dict = data_dict

    def __len__(self)->int:
        return len(self.data_dict)
    
    def __getitem__(self,
                    x:int):
        _data = self.data_dict[x]
        return _data['src']+[self.source_vocab.index('<EOW>')], _data['tgt']+[self.target_vocab.index('<EOW>')]
    
#    def __getitem__(self, x:int):
#        return self.order[x][self.source_ids] + [self.source_vocab.index('<EOW>')], self.order[x][self.target_ids] + [self.target_vocab.index('<EOW>')]
    
        
onechar_dataset = Onechar_dataset(source=source, target=target)
# print(onechar_dataset.__len__())
# print(onechar_dataset.__getitem__(0))

for i in range(onechar_dataset.__len__()):
    inp, tch = onechar_dataset.__getitem__(i)
    print(f'{i:3d}', 
          f'orth:{onechar_dataset.data_dict[i]["orth"]}', 
          f'inp:{inp}, tch:{tch}', 
          f"{colored(onechar_dataset.data_dict[i]['yomi'],'blue',attrs=['bold'])}")

# 2 モデルの定義

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# 自作ライブラリ LAM の読み込み
#import lam 
#from lam import EncoderRNN

class EncoderRNN(nn.Module):
    """RNNによる符号化器"""
    def __init__(self,
            n_inp:int=0,
            n_hid:int=0):
            #device=device):
        super().__init__()
        self.n_hid = n_hid if n_hid != 0 else 8
        self.n_inp = n_inp if n_inp != 0 else 8

        self.embedding = nn.Embedding(n_inp, n_hid)
        self.gru = nn.GRU(n_hid, n_hid)

    def forward(self,
                inp:int=0,
                hid:int=0,
                device=device
               ):
        embedded = self.embedding(inp).view(1, 1, -1)
        out = embedded
        out, hid = self.gru(out, hid)
        return out, hid

    def initHidden(self)->torch.Tensor:
        return torch.zeros(1, 1, self.n_hid, device=device)


#from lam import AttnDecoderRNN
class AttnDecoderRNN(nn.Module):
    """注意付き復号化器の定義"""
    def __init__(self, 
                 n_hid:int=0, 
                 n_out:int=0, 
                 dropout_p:float=0.0, 
                 max_length:int=0):
        super().__init__()
        self.n_hid = n_hid
        self.n_out = n_out
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.n_out, self.n_hid)
        self.attn = nn.Linear(self.n_hid * 2, self.max_length)
        self.attn_combine = nn.Linear(self.n_hid * 2, self.n_hid)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.n_hid, self.n_hid)
        self.out = nn.Linear(self.n_hid, self.n_out)

    def forward(self, 
                inp:int=0, 
                hid:int=0, 
                encoder_outputs:torch.Tensor=None, 
                device=device):
        embedded = self.embedding(inp).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hid[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

        out = torch.cat((embedded[0], attn_applied[0]), 1)
        out = self.attn_combine(out).unsqueeze(0)

        out = F.relu(out)
        out, hid = self.gru(out, hid)

        out = F.log_softmax(self.out(out[0]), dim=1)
        return out, hid, attn_weights

    def initHidden(self)->torch.Tensor:
        return torch.zeros(1, 1, self.n_hid, device=device)

    
def convert_ids2tensor(
    sentence_ids:list, 
    device:torch.device=torch.device("cuda:0" if torch.cuda.is_available () else "cpu")):
    
    """数値 ID リストをテンソルに変換
    例えば，[0,1,2] -> tensor([[0],[1],[2]])
    """
    return torch.tensor(sentence_ids, dtype=torch.long, device=device).view(-1, 1)
    
    
def evaluate(encoder:nn.Module,
             decoder:nn.Module,
             input_ids,
             max_length,
             source_vocab,
             target_vocab,
             source_ids,
             target_ids,
            )->(list,torch.LongTensor):
    with torch.no_grad():
        input_tensor = convert_ids2tensor(input_ids)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.n_hid, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[source_vocab.index('<SOW>')]], device=device)
        decoder_hidden = encoder_hidden

        decoded_words, decoded_ids = [], []  # decoded_ids を追加
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs, device=device)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            decoded_ids.append(int(topi.squeeze().detach())) # decoded_ids に追加
            if topi.item() == target_vocab.index('<EOW>'):
                decoded_words.append('<EOW>')
                break
            else:
                decoded_words.append(target_vocab[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoded_ids, decoder_attentions[:di + 1]  # decoded_ids を返すように変更
        #return decoded_words, decoder_attentions[:di + 1]

    
#from lam import calc_accuracy
def calc_accuracy(
    _dataset,
    encoder,
    decoder,
    max_length=None,
    source_vocab=None,
    target_vocab=None,
    source_ids=None,
    target_ids=None,
    isPrint=False):

    ok_count = 0
    for i in range(_dataset.__len__()):
        _input_ids, _target_ids = _dataset.__getitem__(i)
        _output_words, _output_ids, _attentions = evaluate(
            encoder=encoder,
            decoder=decoder,
            input_ids=_input_ids,
            max_length=max_length,
            source_vocab=source_vocab,
            target_vocab=target_vocab,
            source_ids=source_ids,
            target_ids=target_ids,
        )
        ok_count += 1 if _target_ids == _output_ids else 0
        if (_target_ids != _output_ids) and (isPrint):
            print(i, _target_ids == _output_ids, _output_words, _input_ids, _target_ids)

    return ok_count/_dataset.__len__()


encoder = EncoderRNN(
    len(source_vocab), 
    params['hidden_size']).to(device)

decoder = AttnDecoderRNN(
    n_hid=params['hidden_size'], 
    n_out=len(target_vocab), 
    dropout_p=params['dropout_p'],
    max_length=_vocab.max_length).to(device)
    
if (params['pretrained']) and (params['path_saved'] != False) and os.path.exists(params['path_saved']):
    """セーブした学習済のモデルがあれば読み込む"""
    
    checkpoint = torch.load(params['path_saved'])
    encoder.load_state_dict(checkpoint['encoder'])
    decoder.load_state_dict(checkpoint['decoder'])
    encoder.eval()
    decoder.eval()
    print(colored(f"セーブした学習済のモデル {params['path_saved']} があるので読み込みました",
          color='blue', attrs=['bold']))

    
# モデルの概要を印字
print(f'encoder:{encoder}')
print(f'decoder:{decoder}')
        
for test_name, val_dataset in X_vals.items():
    acc = calc_accuracy(_dataset=val_dataset,
                        encoder=encoder,
                        decoder=decoder,
                        max_length=_vocab.max_length,
                        source_vocab=source_vocab,
                        target_vocab=target_vocab,
                        source_ids=source_ids,
                        target_ids=target_ids)
    print(colored(f'{test_name} の精度:{acc:.3f}','blue', attrs=['bold']))


# params の印刷
print(colored(params,'blue',attrs=['bold']))    

### 2.1 `_train`, `_fit` など下請け関数の定義

In [ ]:
import math
import random
import numpy as np
import time

def asMinutes(s:int)->str:
    """時間変数を見やすいように，分と秒に変換して返す"""
    m = math.floor(s / 60)
    s -= m * 60
    return f'{int(m):2d}分 {int(s):2d}秒'
    return '%dm %ds' % (m, s)


def timeSince(since:time.time,
            percent:time.time)->str:
    """開始時刻 since と，現在の処理が全処理中に示す割合 percent を与えて，経過時間と残り時間を計算して表示する"""
    now = time.time()  #現在時刻を取得
    s = now - since    # 開始時刻から現在までの経過時間を計算
    #s = since - now
    es = s / (percent) # 経過時間を現在までの処理割合で割って終了予想時間を計算
    rs = es - s        # 終了予想時刻から経過した時間を引いて残り時間を計算

    return f'経過時間:{asMinutes(s)} (残り時間 {asMinutes(rs)})'


def check_vals_performance(encoder=None, 
                           decoder=None,
                           _dataset=None,
                           max_length=0,
                           source_vocab=None, 
                           target_vocab=None,
                           source_ids=None, 
                           target_ids=None):

    if _dataset == None or encoder == None or decoder == None or max_length == 0 or source_vocab == None:
        return
    print('検証データ:',end="")
    for _x in _dataset:
        ok_count = 0
        for i in range(_dataset[_x].__len__()):
            _input_ids, _target_ids = _dataset[_x].__getitem__(i)
            _output_words, _output_ids, _attentions = evaluate(encoder, 
                                                               decoder, 
                                                               _input_ids,
                                                               max_length,
                                                               source_vocab=source_vocab, 
                                                               target_vocab=target_vocab,
                                                               source_ids=source_ids, 
                                                               target_ids=target_ids)
            ok_count += 1 if _target_ids == _output_ids else 0
        print(f'{_x}:{ok_count/_dataset[_x].__len__():.3f},',end="")
    print()


def _train(input_tensor:torch.Tensor=None, 
           target_tensor:torch.Tensor=None,
           encoder:torch.nn.Module=None, 
           decoder:torch.nn.Module=None,
           encoder_optimizer:torch.optim=None, 
           decoder_optimizer:torch.optim=None,
           criterion:torch.nn.modules.loss=torch.nn.modules.loss.CrossEntropyLoss,
           max_length:int=_vocab.max_length,
           target_vocab:list=None,
           teacher_forcing_ratio:float=0.,
           device:torch.device=device)->float:
    
    """inpute_tensor (torch.Tensor() に変換済の入力系列) を 1 つ受け取って，
    encoder と decoder の訓練を行う
    """
    
    encoder_hidden = encoder.initHidden() # 符号化器の中間層を初期化
    encoder_optimizer.zero_grad()         # 符号化器の最適化関数の初期化
    decoder_optimizer.zero_grad()         # 復号化器の最適化関数の初期化

    input_length = input_tensor.size(0)   # 0 次元目が系列であることを仮定
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.n_hid, device=device)
    
    loss = 0.  # 損失関数値
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            inp=input_tensor[ei], 
            hid=encoder_hidden, 
            device=device)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[target_vocab.index('<SOW>')]], device=device)
    decoder_hidden = encoder_hidden

    ok_flag = True
    # 教師強制をするか否かを確率的に決める
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing: # 教師強制する場合 Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, 
                                                                        decoder_hidden, 
                                                                        encoder_outputs,
                                                                        device=device)
            decoder_input = target_tensor[di]      # 教師強制 する
            
            loss += criterion(decoder_output, target_tensor[di])
            ok_flag = (ok_flag) and (decoder_output.argmax() == target_tensor[di].detach().numpy()[0])
            if decoder_input.item() == target_vocab.index('<EOW>'):
                break

    else: # 教師強制しない場合 Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, 
                                                                        decoder_hidden, 
                                                                        encoder_outputs,
                                                                        device=device)
            topv, topi = decoder_output.topk(1)     # 教師強制しない
            decoder_input = topi.squeeze().detach() 

            loss += criterion(decoder_output, target_tensor[di])
            ok_flag = (ok_flag) and (decoder_output.argmax() == target_tensor[di].detach().numpy()[0])
            if decoder_input.item() == target_vocab.index('<EOW>'):
                break

    loss.backward()           # 誤差逆伝播
    encoder_optimizer.step()  # encoder の学習
    decoder_optimizer.step()  # decoder の学習
    return loss.item() / target_length, ok_flag


def _fit(encoder:torch.nn.Module, 
         decoder:torch.nn.Module,
         epochs:int=1,
         lr:float=0.0001,
         n_sample:int=3,
         teacher_forcing_ratio=False,
         train_dataset:torch.utils.data.Dataset=None,
         val_dataset:dict=None,
         source_vocab:list=None,
         target_vocab:list=None,
         source_ids:str=None,
         target_ids:list=None,
         params:dict=None,
         max_length:int=1,
         device=device,
        )->list:

    start_time = time.time()

    encoder.train()
    decoder.train()
    encoder_optimizer = params['optim_func'](encoder.parameters(), lr=lr)
    decoder_optimizer = params['optim_func'](decoder.parameters(), lr=lr)
    criterion = params['loss_func']
    losses = []

    for epoch in range(epochs):
        epoch_loss = 0
        ok_count = 0
        
        #エポックごとに学習順をシャッフルする
        learning_order = np.random.permutation(train_dataset.__len__())
        
        for i in range(train_dataset.__len__()):
            x = learning_order[i]   # ランダムにデータを取り出す
            input_ids, target_ids = train_dataset.__getitem__(x)
            input_tensor = convert_ids2tensor(input_ids)
            target_tensor = convert_ids2tensor(target_ids)

            #訓練の実施
            loss, ok_flag = _train(input_tensor=input_tensor, 
                                   target_tensor=target_tensor,
                                   encoder=encoder, 
                                   decoder=decoder,
                                   encoder_optimizer=encoder_optimizer, 
                                   decoder_optimizer=decoder_optimizer,
                                   criterion=criterion,
                                   max_length=max_length,
                                   target_vocab=target_vocab,
                                   teacher_forcing_ratio=teacher_forcing_ratio,
                                   device=device)
            epoch_loss += loss
            ok_count += 1 if ok_flag else 0


        losses.append(epoch_loss/train_dataset.__len__())
        print(colored(f'エポック:{epoch:2d} 損失:{epoch_loss/train_dataset.__len__():.2f}', 'blue', attrs=['bold']),
              colored(f'{timeSince(start_time, (epoch+1) * train_dataset.__len__()/(epochs * train_dataset.__len__()))}',
                      'cyan', attrs=['bold']),
              colored(f'訓練データの精度:{ok_count/train_dataset.__len__():.3f}', 'blue', attrs=['bold']))

        check_vals_performance(_dataset=val_dataset,
                               encoder=encoder,
                               decoder=decoder,
                               max_length=max_length,
                               source_vocab=source_vocab,
                               target_vocab=target_vocab,
                               source_ids=source_ids,
                               target_ids=target_ids)
        if n_sample > 0:
            evaluateRandomly(encoder, decoder, n=n_sample)

    return losses

## 2.2 一文字データの学習

In [ ]:
params

In [ ]:
losses = []
losses += _fit(encoder=encoder, 
               decoder=decoder, 
               device=device,
               epochs=10,
               #epochs=params['epochs'], 
               max_length=_vocab.max_length,
               n_sample=0,
               params=params,
               source_vocab=source_vocab,
               target_vocab=target_vocab,
               source_ids=source_ids,
               target_ids=target_ids,
               teacher_forcing_ratio=params['teacher_forcing_ratio'],
               #train_dataset=train_dataset,
               train_dataset=onechar_dataset,
               #lr=params['lr'],
               lr=0.001,
               val_dataset=None,
               #val_dataset=X_vals,
              )

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.show()

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!ls -lt *.pt

In [ ]:
X = torch.load('2023_0123hoge.pt')
enocder=X['encoder']
decoder=X['decoder']

In [ ]:
counter = 1
for k1 in fushimi1999.keys():
    k1_old = ""
    for w1 in fushimi1999[k1]:
        ans=evaluate(encoder,
                     decoder,
                     [_vocab.orth_vocab.index(w1[0]),_vocab.orth_vocab.index(w1[1]),1],
                     34,
                     source_vocab,
                     target_vocab,
                     source_ids,
                     target_ids)
        if k1 != k1_old:
            k1_old = k1
            print(colored(f'{k1}:', 'cyan', attrs=['bold']), end="\n")
        counter =  1 if (counter % 10) == 0 else (counter + 1)
        _end = "\n" if counter==1 else ", "
        print(f'{w1}:{"".join(p for p in ans[0][:-1])}', end=_end)

## 2.3 訓練データセットの定義

In [ ]:
# 訓練データセットと検証データセットを作成
train_dataset = Train_dataset(data=_vocab.train_data,
                              source_vocab=source_vocab, 
                              target_vocab=target_vocab,
                              source_ids=source_ids,   # おそらくこの 2 行を入れないといけなかった
                              target_ids=target_ids)  # そうでなければ，デフォルトの `mora_p_r` になってしまう

P  = int(train_dataset.__len__() * 0.9)
_P = train_dataset.__len__() - P
_train_dataset, _val_dataset = torch.utils.data.random_split(dataset=train_dataset,
                                                            lengths=(P, _P),
                                                            generator=torch.Generator().manual_seed(42))

                
# 訓練データセットと検証データセットを作成
print(f'len(_train_dataset):{len(_train_dataset)}',
      f'len(_val_dataset):{len(_val_dataset)}')      

# 3 本来のデータセットの学習

In [ ]:
losses = []
losses += _fit(encoder=encoder, 
               decoder=decoder, 
               device=device,
               #epochs=1,
               epochs=params['epochs'], 
               max_length=_vocab.max_length,
               n_sample=0,
               params=params,
               source_vocab=source_vocab,
               target_vocab=target_vocab,
               source_ids=source_ids,
               target_ids=target_ids,
               teacher_forcing_ratio=params['teacher_forcing_ratio'],
               train_dataset=_train_dataset,
               #train_dataset=onechar_dataset,
               lr=params['lr'],
               #val_dataset=None,
               val_dataset={'_val':_val_dataset}
              )

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.show()

## 3.1 訓練済モデルの保存

In [ ]:
print(params)
torch.save({'encoder': encoder,'decoder': decoder, 'params':params}, '2023_0123lam_o2p_hid64.pt')

In [ ]:
def check_fushimi1999_list(encoder=encoder,
                       decoder=decoder):

    counter = 1
    for k1 in fushimi1999.keys():
        k1_old = ""
        for w1 in fushimi1999[k1]:
            ans=evaluate(encoder,
                         decoder,
                         [_vocab.orth_vocab.index(w1[0]),_vocab.orth_vocab.index(w1[1]),1],
                         _vocab.max_length,
                         source_vocab,
                         target_vocab,
                         source_ids,
                         target_ids)
            if k1 != k1_old:
                k1_old = k1
                print(colored(f'{k1}:', 'cyan', attrs=['bold']), end="\n")
            counter =  1 if (counter % 10) == 0 else (counter + 1)
            _end = "\n" if counter==1 else ", "
            print(f'{w1}',
                  colored(f'/{"".join(p for p in ans[0][:-1])}/','grey', attrs=['bold']), end=_end)

            
check_fushimi1999_list()    